<a href="https://colab.research.google.com/github/lebryant/test_repo/blob/master/NYSETrial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
import os


KeyboardInterrupt: ignored

In [0]:
import os
import pandas as pd
import statsmodels.api as sm
import statsmodels.tsa.filters.bk_filter
import matplotlib.pyplot as plt

In [0]:
ts_data = pd.read_csv("prices.csv")
ts_data.head()

ts_data['date'] = pd.to_datetime(ts_data['date'])
ts_data = ts_data.set_index('date')
ts_data.index



In [0]:
y = ts_data['close']
y.plot()
plt.show()

In [0]:
from pylab import rcParams
rcParams['figure.figsize'] = 18, 8
decomposition = sm.tsa.filters.bkfilter(y, low = 6*90, high = 32*90)
fig = decomposition.plot()
plt.show()

In [0]:
ts_data.head()

In [0]:
funda_data = pd.read_csv("fundamentals.csv")
funda_data.head()
price_data = pd.read_csv("prices.csv")
price_data.head()
model_data = pd.DataFrame(funda_data.set_index('Ticker Symbol').join(price_data.set_index('symbol')))
model_data.reset_index(level=0, inplace=True)
model_data.reset_index(level=0, inplace=True)

In [0]:
aal_data = model_data[model_data['index'] == 'AAL']
aal_close = pd.Series(model_data[model_data['index'] == 'AAL']['close']).shift(1)
aal_data['prev_close'] = aal_close
aal_data.drop(['level_0','index','Unnamed: 0','Period Ending','volume'], axis = 1, inplace = True)
aal_data.head()
aal_data_2 = model_data[model_data['index'] == 'AAL'].shift(1).drop('close', axis=1)
aal_data_2['close'] = model_data[model_data['index'] == 'AAL']['close']
aal_data_2['prev_close'] = model_data[model_data['index'] == 'AAL']['close'].shift(1)
aal_data_2.head()

In [0]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
import sklearn.metrics 

aal_data_2.dropna(inplace=True)
X = aal_data_2.drop(['close','date','open','low','high','Period Ending','level_0','index','Unnamed: 0'], axis = 1)
Y = aal_data_2['close']

X.head()

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=23)

test_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=1, random_state=0, loss='ls').fit(X_train, Y_train)
sklearn.metrics.r2_score(Y_test, test_model.predict(X_test)) 

Y_pred = pd.DataFrame(test_model.predict(X_test))

pred_both = Y_pred.join(Y_test).dropna()
pred_both.columns = ['pred','act']
pred_both.pred.plot()
pred_both.act.plot()

plt.show()